# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

In [2]:
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object

In [3]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

# 1. Rerefence genome data preparation
## 1.1. Check reference genome installation

Celloracle uses genomepy to get DNA sequence data. Before starting celloracle analysis, we need to make sure that the reference genome is correctly installed in your computational environment. If not, please install reference genome.

In [4]:
# PLEASE make sure that you are setting correct ref genome.
ref_genome = "mm10"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
print(ref_genome, "installation: ", genome_installation)

mm10 installation:  True


## 1.2. Install reference genome (if refgenome is not installed)

In [5]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(ref_genome, "UCSC")
else:
    print(ref_genome, "is installed.")

mm10 is installed.


# 2. Load data

## 2.1. Load processed peak data 

In [6]:
# Load annotated peak data.
peaks = pd.read_parquet("../01_ATAC-seq_data_processing/option1_scATAC-seq_data_analysis_with_cicero/peak_file.parquet")
peaks.head()

peak_id gene_short_name
0  chr10_100015291_100017830            Kitl
1  chr10_100486534_100488209           Tmtc3
2  chr10_100588641_100589556   4930430F08Rik
3  chr10_100741247_100742505         Gm35722
4  chr10_101681379_101682124          Mgat4c

## 2.1. Check data

In [7]:
# Define function for quality check
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'
        
    Returns:
        tuple: chromosome name, start position, end position
    """
    
    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_foamat(peaks_df, ref_genome):
    """
    Check peak fomat. 
     (1) Check chromosome name. 
     (2) Check peak size (length) and remove sort DNAs (<5bp)
    
    """
    
    df = peaks_df.copy()
    
    n_peaks_before = df.shape[0]
    
    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed))
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(np.int)
    df_decomposed["end"] = df_decomposed["end"].astype(np.int)
    
    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())
    
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    
    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]
    
    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)
    
    return df

In [8]:
peaks = check_peak_foamat(peaks, ref_genome)

Peaks before filtering:  15680
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  15680


## 2.2. [Optional step] Load motifs
You can select TF binding motif data for Celloracle motif analysis.
If you have no preference and just want to use a default motif, you can skip this step.
If you want to use a non-default motif dataset, please prepare motif data as a list of motif class in gimmemotifs.
We have several option for loading motif database as below.
    
### 2.2.1. [Optional step] Load motif data from gimmemotifs dataset

Many motif databases are included with GimmeMotifs. https://gimmemotifs.readthedocs.io/en/master/overview.html
You can load them as follows.

In [9]:
# First, we need to pick up motifs for your dataset. We can get a list.

# Get folder path that stores motif data.
import os, glob
from gimmemotifs.motif import MotifConfig
config = MotifConfig()
motif_dir = config.get_motif_dir()

# Get list for motif data name
motifs_data_name = [i for i in os.listdir(motif_dir) if i.endswith(".pfm")]
motifs_data_name.sort()
motifs_data_name

['CIS-BP.pfm',
 'ENCODE.pfm',
 'HOCOMOCOv10_HUMAN.pfm',
 'HOCOMOCOv10_MOUSE.pfm',
 'HOCOMOCOv11_HUMAN.pfm',
 'HOCOMOCOv11_MOUSE.pfm',
 'HOMER.pfm',
 'IMAGE.pfm',
 'JASPAR2018.pfm',
 'JASPAR2018_fungi.pfm',
 'JASPAR2018_insects.pfm',
 'JASPAR2018_nematodes.pfm',
 'JASPAR2018_plants.pfm',
 'JASPAR2018_urochordates.pfm',
 'JASPAR2018_vertebrates.pfm',
 'JASPAR2020.pfm',
 'JASPAR2020_fungi.pfm',
 'JASPAR2020_insects.pfm',
 'JASPAR2020_nematodes.pfm',
 'JASPAR2020_plants.pfm',
 'JASPAR2020_urochordates.pfm',
 'JASPAR2020_vertebrates.pfm',
 'RSAT_insects.pfm',
 'RSAT_plants.pfm',
 'RSAT_vertebrates.pfm',
 'SwissRegulon.pfm',
 'factorbook.pfm',
 'gimme.vertebrate.v5.0.pfm']

In [10]:
# Once you picked up motifs, you can load the motif files with "read_motifs"
from gimmemotifs.motif import read_motifs

path = os.path.join(motif_dir, "JASPAR2018_plants.pfm")# Please enter motifs name here 
motifs = read_motifs(path)

# Check first 10 motifs
motifs[:10]

[MA0020.1_Dof2_AAAGCn,
 MA0021.1_Dof3_AAAGyn,
 MA0034.1_Gam1_nnyAACCGmC,
 MA0044.1_HMG-1_sTTGTnyTy,
 MA0045.1_HMG-I/Y_nwAnAAAnrnmrAmAy,
 MA0053.1_MNB1A_AAAGC,
 MA0054.1_myb.Ph3_TAACnGTTw,
 MA0064.1_PBF_AAAGy,
 MA0082.1_squamosa_mCAwAwATrGwAAn,
 MA0096.1_bZIP910_mTGACGT]

### 2.2.2 [Optional step] Load motif data from celloracle motif dataset

Celloracle also provides many motif dataset that was generated from CisBP. http://cisbp.ccbr.utoronto.ca/

These motifs were organized by each species. Please select motifs for your species.

If you have a request for motifs for a new species, you can ask us to add new motifs through github issue page.

In [11]:
# Check available motifs
ma.MOTIFS_LIST

['CisBP_ver2_Arabidopsis_thaliana.pfm',
 'CisBP_ver2_Caenorhabditis_elegans.pfm',
 'CisBP_ver2_Danio_rerio.pfm',
 'CisBP_ver2_Drosophila_ananassae.pfm',
 'CisBP_ver2_Drosophila_erecta.pfm',
 'CisBP_ver2_Drosophila_grimshawi.pfm',
 'CisBP_ver2_Drosophila_melanogaster.pfm',
 'CisBP_ver2_Drosophila_mix.pfm',
 'CisBP_ver2_Drosophila_mojavensis.pfm',
 'CisBP_ver2_Drosophila_persimilis.pfm',
 'CisBP_ver2_Drosophila_pseudoobscura.pfm',
 'CisBP_ver2_Drosophila_sechellia.pfm',
 'CisBP_ver2_Drosophila_simulans.pfm',
 'CisBP_ver2_Drosophila_virilis.pfm',
 'CisBP_ver2_Drosophila_willistoni.pfm',
 'CisBP_ver2_Drosophila_yakuba.pfm',
 'CisBP_ver2_Homo_sapiens.pfm',
 'CisBP_ver2_Mus_musculus.pfm',
 'CisBP_ver2_Rattus_norvegicus.pfm',
 'CisBP_ver2_Saccharomyces_cerevisiae.pfm',
 'CisBP_ver2_Xenopus_laevis.pfm',
 'CisBP_ver2_Xenopus_tropicalis.pfm',
 'CisBP_ver2_Xenopus_tropicalis_and_Xenopus_laevis.pfm']

In [12]:
# Load motifs from celloracle dataset.
motifs = ma.load_motifs("CisBP_ver2_Mus_musculus.pfm")

# Check first 10 motifs
motifs[:10]

[M00008_2.00_nnnAAww,
 M00044_2.00_nrTAAACAn,
 M00056_2.00_TAATAAAT,
 M00060_2.00_nnnTTCnnn,
 M00111_2.00_nGCCynnGGs,
 M00112_2.00_CCTsrGGCnA,
 M00113_2.00_nsCCnnAGGs,
 M00114_2.00_nnGCCynnGG,
 M00115_2.00_nnATnAAAn,
 M00116_2.00_nnAATATTAnn]

# 3. Instantiate TFinfo object and search for TF binding motifs
The motif analysis module has a custom class; TFinfo. The TFinfo object converts a peak data into a DNA sequences and scans the DNA sequences searching for TF binding motifs. Then, the results of motif scan will be filtered and converted into either a python dictionary or a depending on your preference. This TF information is necessary for GRN inference.

## 3.1. Instantiate TFinfo object

In [16]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks, # peak info calculated from ATAC-seq data
                       ref_genome=ref_genome) 

## 3.2. Motif scan


!!You can set TF binding motif information as an argument: 

tfi.scan(motifs=motifs)

If you don't set motifs or set None, default motifs will be loaded automatically.

- For mouse and human, "gimme.vertebrate.v5.0." will be used as a default motifs. 
- For another species, a species specific TF binding motif data extracted from CisBP ver2.0 will be used.



In [ ]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02, 
            motifs=None,  # If you enter None, default motifs will be loaded.
            verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="test1.celloracle.tfinfo")

In [16]:
# Check motif scan results
tfi.scanned_df.head()

seqname                      motif_id factors_direct  \
0  chr10_100015291_100017830       GM.5.0.Homeodomain.0001          TGIF1   
1  chr10_100015291_100017830             GM.5.0.Mixed.0001                  
2  chr10_100015291_100017830             GM.5.0.Mixed.0001                  
3  chr10_100015291_100017830             GM.5.0.Mixed.0001                  
4  chr10_100015291_100017830  GM.5.0.Nuclear_receptor.0002          NR2C2   

         factors_indirect      score   pos  strand  
0  ENSG00000234254, TGIF1  10.311002  1003       1  
1               SRF, EGR1   7.925873   481       1  
2               SRF, EGR1   7.321375   911      -1  
3               SRF, EGR1   7.276585   811      -1  
4            NR2C2, Nr2c2   9.067331   449      -1

We have the score for each sequence and motif_id pair.
In the next step we will filter the motifs with low score.

# 4. Filtering motifs

In [15]:
# Reset filtering 
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10.5)

# Do post filtering process. Convert results into several file format.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)



peaks were filtered: 12952283 -> 2288874
1. converting scanned results into one-hot encoded dataframe.



2. converting results into dictionaries.
converting scan results into dictionaries...


# 5. Get Final results

## 5.1. Get resutls as a dictionary

In [23]:
td = tfi.to_dictionary(dictionary_type="targetgene2TFs")


## 5.2. Get results as a dataframe

In [17]:
df = tfi.to_dataframe()
df.head()

peak_id gene_short_name  9430076c15rik  Ac002126.6  \
0  chr10_100015291_100017830            Kitl            0.0         0.0   
1  chr10_100486534_100488209           Tmtc3            0.0         0.0   
2  chr10_100588641_100589556   4930430F08Rik            0.0         0.0   
3  chr10_100741247_100742505         Gm35722            0.0         0.0   
4  chr10_101681379_101682124          Mgat4c            0.0         0.0   

   Ac012531.1  Ac226150.2  Afp  Ahr  Ahrr  Aire  ...  Znf784  Znf8  Znf816  \
0         0.0         0.0  0.0  1.0   1.0   0.0  ...     0.0   0.0     0.0   
1         0.0         0.0  0.0  0.0   0.0   0.0  ...     1.0   0.0     0.0   
2         1.0         0.0  0.0  1.0   1.0   0.0  ...     0.0   0.0     0.0   
3         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   
4         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   

   Znf85  Zscan10  Zscan16  Zscan22  Zscan26  Zscan31  Zscan4  
0    0.0      0.0      0.0      0.0      0.0      1.0     0.0  
1    0.0      0.0      0.0      1.0      0.0      0.0     0.0  
2    0.0      0.0      0.0      0.0      0.0      0.0     0.0  
3    0.0      0.0      0.0      0.0      0.0      0.0     0.0  
4    0.0      0.0      0.0      0.0      0.0      0.0     1.0  

[5 rows x 1092 columns]

# 6. Save TFinfo as dictionary or dataframe
We'll use this information when making the GRNs. Save the results.

In [19]:
folder = "TFinfo_outputs"
os.makedirs(folder, exist_ok=True)

# save TFinfo as a dictionary
td = tfi.to_dictionary(dictionary_type="targetgene2TFs")
save_as_pickled_object(td, os.path.join(folder, "TFinfo_targetgene2TFs.pickled"))

# save TFinfo as a dataframe
df = tfi.to_dataframe()
df.to_parquet(os.path.join(folder, "TFinfo_dataframe.parquet"))